In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

<h3 style="color:green;">SCREENSHOT OF RUNNING SCENE IN FACTORY IO</h3>
<p>Factory I/O is a 3D factory simulation for learning automation technologies. Designed to be easy to use, it allows to quickly build a virtual factory using a selection of common industrial parts.

Factory I/O also includes many scenes inspired by typical industrial applications, ranging from beginner to advanced difficulty levels.

The most common scenario is to use Factory I/O as a PLC training platform since PLC are the most common controllers found in industrial applications. However, it can also be used with microcontrollers, SoftPLC, Modbus, among many other technologies.</p>

<a href="url">https://factoryio.com/</a>

![title](images/Conveyor.PNG)

<h3 style="color:green;">SCREENSHOT OF CONTROL IO CONFIGURATOR</h3>
<p>CONTROL I/O is an easy to use, brand independent SoftPLC, designed from the ground up for FACTORY I/O.<br> The goal of CONTROL I/O is to provide a simple and intuitive experience for anyone starting in the world of automation.</p>
<a href="url">https://docs.factoryio.com</a>

![title](images/Control_IO.PNG)

<h3 style="color:green;">Finding the ideal range of time between boxes using idealistic data<h3>
<ol style="font-weight: normal;">
<li>Some point</li>
<li>Another point</li>
<li>More points if required</li>
</ol>


In [ ]:
#Importing the csv file from factory io

#Commenting out block of code for local machine usage
#data = pd.read_csv('C:/Users/a.sethu/Desktop/Python/normal.csv')
#df = pd.DataFrame(data)

#Importing the csv file created in factory io
#File currently contains 6 Columns and 682,211 Rows
import zipfile
from zipfile import ZipFile
file_name = "data/normal.zip"
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()

In [ ]:
#Subseting all Light Array Emitter data
filteredLAE = df[df[' Name']=='Light Array Emitter 1 (Value)']
#using a copy to prevent index warning
filteredLAE = filteredLAE.copy()
#converting the current value column to numeric
filteredLAE[' Current Value'] = pd.to_numeric(filteredLAE[' Current Value'])

In [ ]:
#using a copy to prevent index warning
filteredLAE = filteredLAE.copy()
filteredLAE[' Sensor Value Difference'] = filteredLAE[' Current Value'].diff()
filteredLAE

In [ ]:
#Filtering sensor trigger data points from the data by removing 
#all all data with current value = 0
filteredLAE = filteredLAE[filteredLAE[' Sensor Value Difference'] != 0]
filteredLAE = filteredLAE[filteredLAE[' Current Value']!=128 ]
filteredLAE = filteredLAE[filteredLAE[' Current Value']!=64 ]
#Dropping al NaN values
filteredLAE.dropna(subset = [' Sensor Value Difference'], inplace=True)
filteredLAE

In [ ]:
filteredLAE[filteredLAE[' Sensor Value Difference'] > 0]
filteredLAE['Trigger Status']="Off"
filteredLAE['Trigger Status'] = filteredLAE.apply(lambda x: 'On' if x[' Sensor Value Difference']>0 else 'Off',axis=1)

In [ ]:
filteredLAE['Trigger Interval'] = filteredLAE[" Time Elapsed"].diff(1)

In [ ]:
preprocessedDF = filteredLAE[filteredLAE['Trigger Status']=='On']

In [ ]:
#using a copy to prevent index warning
preprocessedDF = preprocessedDF.copy()
preprocessedDF['Trigger Interval'] = preprocessedDF[" Time Elapsed"].diff(1)
preprocessedDF.dropna(subset = ['Trigger Interval'], inplace=True)
preprocessedDF

In [ ]:
import seaborn as sns
preprocessedDF=preprocessedDF.reset_index(drop=True)
plt.figure(figsize=(12,8)) #Plotting the data points on the plot
sns.stripplot(x='Trigger Interval',data = preprocessedDF,jitter= True,edgecolor = 'None')
sns.despine
locs,labels = plt.xticks()
plt.yticks([])

<h3 style="color:green;">Using Jenks Natural Optimisation Breaks for finding the range <h3>
<ol style="font-weight: normal;">
<li>Some point</li>
<li>Another point</li>
<li>More points if required</li>
</ol>

In [ ]:
import jenkspy
#Finding the breaks using the Jenks natural breaks optimzation method
breaks = jenkspy.jenks_breaks(preprocessedDF['Trigger Interval'],nb_class = 3)

In [ ]:
breaks

In [ ]:
import pickle
filename = 'data/finalized_model.sav'
pickle.dump(breaks, open(filename, 'wb'))
 
# some time later...
 


In [ ]:
#seaborn library for plotting
import seaborn as sns
plt.figure(figsize=(12,8)) #Plotting the data points on the plot
sns.stripplot(x='Trigger Interval',data = preprocessedDF,jitter= True,edgecolor = 'None')
sns.despine
locs,labels = plt.xticks()
plt.yticks([])
#Drawing vertical lines usng the data points as x values
for b in breaks:
    plt.vlines(b,ymin= -0.2,ymax=0.2,color='k')

<h3 style="color:green;">The boxplot shows the ideal range lies between 4.2 to 6.2 </h3>

In [ ]:
#Box plot
import numpy as np
sns.boxplot(x='Trigger Interval', data=preprocessedDF, whis=np.inf)

In [ ]:
array_list = []
B = plt.boxplot(preprocessedDF['Trigger Interval'])
[array_list.append(item.get_ydata()) for item in B['whiskers']]
download_list = []
for item in array_list:
    download_list.append(item[0])

In [ ]:
import pickle
filename = 'data/finalized_model.sav'
pickle.dump(download_list, open(filename, 'wb'))